In [1]:
import pandas as pd
import os
from PIL import Image
if os.getcwd().split(os.sep)[-1] == 'notebooks':
    os.chdir('..') # set/ ccwd as the parent directory to make imports easier
import json
import ast
import re

from serve.utils_llm import get_llm_output

In [2]:
df = pd.read_csv('results.csv')
# Step 1: Split the 'prompt' into 'question' and 'answers'
df['question'] = df['prompt'].apply(lambda x: x.split("\nModel A: ")[0])
df['answers'] = df['prompt'].apply(lambda x: x.split("\nModel A: ")[1])

# Step 2: Further split 'answers' into 'Model A' and 'Model B'
df['answer_a'] = df['answers'].apply(lambda x: x.split("\nModel B: ")[0].strip("[]").replace("'", ""))
df['answer_b'] = df['answers'].apply(lambda x: x.split("\nModel B: ")[1].strip("[]").replace("'", ""))

# Clean up
df.drop('answers', axis=1, inplace=True)  # Remove the intermediate 'answers' column

In [4]:
# Regular expression to match entities: Capitalized words or phrases followed by a colon
# regex_pattern = r'-\s*([A-Za-z ]+):'
regex_pattern = r'-\s*(?:\*\*)?([A-Za-z ]+?)(?:\*\*)?:'

def extract_entities(text):
    matches = re.findall(regex_pattern, text)
    return [m for m in matches if m not in ["Model A", "Model B"]]

df['entities'] = df['response'].apply(extract_entities)

smaller_systems_prompt = "You are a helpful assistant. Your outputs adhere to the format given by the user."

In [5]:
# print(df[df['entities'].apply(len) == 0].iloc[0]['response'])
all_entities = set([e for entities in df['entities'] for e in entities])

def extract_axis_descriptions(text):

    lines = text.strip().split('\n')

    # Initialize variables to construct the sections
    sections = []
    current_section = []

    # Process each line, building sections while excluding model scores
    for line in lines:
        # Check if the line starts a new section or is part of the current one
        if line.startswith('- ') and current_section:  # If starting a new section and there is a current section
            # Join the current section lines and add to sections
            sections.append('\n'.join(current_section).strip().replace("- ", "").replace("\n", ""))
            current_section = [line]  # Start a new section
        elif "Model A Score" not in line and "Model B Score" not in line:
            # If the line is not a model score, add it to the current section
            current_section.append(line)

    # Don't forget to add the last section
    if current_section:
        sections.append('\n'.join(current_section).strip().replace("- ", "").replace("\n", ""))
    return sections

df['axis_description'] = df['axis_response'].apply(extract_axis_descriptions)
df.head()

,prompt,response,axes,axis_response,question,answer_a,answer_b,entities,axis_description
0,Is it unreasonable to double your investment y...,- Complexity of language: \n - Model A uses m...,"['Complexity of language', 'Specificity of exa...","- Complexity of language:\n High: Formal, t...",Is it unreasonable to double your investment y...,I know it may not last longer but i was able t...,It is not unreasonable to aim for a high rate ...,"[Complexity of language, Specificity of exampl...","[Complexity of language: High: Formal, tech..."
1,How a computer randomly generates something Ho...,- Complexity of Explanation: Model A provides ...,"['Complexity of Explanation', 'Emphasis on Pre...",- Complexity of Explanation:\n High: Techni...,How a computer randomly generates something Ho...,"""Computers have no imagination , they just do ...","To generate a random link, the computer uses a...","[Complexity of Explanation, Emphasis on Predic...",[Complexity of Explanation: High: Technical...
2,"Q. What can cause panic attacks?Hi doctor,I ha...",- Coverage of Information: Model B provides a ...,"['Coverage of Information', 'Engagement with t...",- Coverage of Information:\n High: Detailed...,"Q. What can cause panic attacks?Hi doctor,I ha...",Hi. For further information consult a psychiat...,There are several potential causes of panic at...,"[Coverage of Information, Engagement with the ...",[Coverage of Information: High: Detailed in...
3,The differences between a liberal and a conser...,- **Scope of Explanation**: \n - **Model A**:...,[],- Scope of Explanation:\n High: Detailed an...,The differences between a liberal and a conser...,The term liberal used to be synonymous with li...,"""Liberal and conservative are two political id...","[Scope of Explanation, Level of Detail, Tone a...",[Scope of Explanation: High: Detailed analy...
4,Why does bright light ( i.e. a flashlight ) ca...,- Depth of Explanation: Model A provides a mor...,"['Depth of Explanation', 'Biological Terminolo...","- Depth of Explanation:\n High: Detailed, l...",Why does bright light ( i.e. a flashlight ) ca...,"""The iris controls the amount of light that en...","When you shine a bright light in your eyes, it...","[Depth of Explanation, Biological Terminology ...","[Depth of Explanation: High: Detailed, laye..."


In [6]:
all_axis_descriptions = set([e for entities in df['axis_description'] for e in entities])
dup_axis_descriptions = [e for entities in df['axis_description'] for e in entities]
len(all_axis_descriptions), len(dup_axis_descriptions)

(333, 333)

In [7]:
all_axis_descriptions

{'Accessibility:    High: Relatable examples, everyday scenarios.    Low: Focus on anatomy and physiology.',
 'Actionability:    High: Urgency for medical attention    Low: General advice',
 'Actionable Advice:    High: Explicit instructions, specific steps    Low: Considerations without instructions',
 'Adaptability:    High: Customizable and flexible    Low: Predefined rules',
 "Additional Information:    High: Includes extra context    Low: Doesn't expand beyond query",
 'Advice Clarity:    High: Direct, specific guidance    Low: Lacks clear advice',
 'Analogical Approach:    High: Uses analogies, simple language.    Low: Relies on technical terms.',
 'Analogical Reasoning:    High: Heavy analogies and visualizations used.    Low: Direct factual language employed.',
 'Analogies Used:    High: Relatable everyday examples, vivid analogies    Low: Listing potential factors, straightforward explanation',
 'Analogies and Comparisons:    High: Uses analogies for clarity.    Low: Presents 

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np

axes_descriptions = all_axis_descriptions
# Extract just the descriptions for embeddings
descriptions = [desc.split(": ", 1)[1] for desc in axes_descriptions]
print(descriptions)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each description
embeddings = model.encode(descriptions)

# Cluster the embeddings
n_clusters = 10  # Adjust based on your preference or use methods like Elbow to find the optimal number
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(embeddings)

# Assign each description to a cluster
clusters = kmeans.labels_

# Group axes by cluster
grouped_axes = {i: [] for i in range(n_clusters)}
for axis, cluster in zip(axes_descriptions, clusters):
    grouped_axes[cluster].append(axis)

# Print grouped axes
for cluster, axes in grouped_axes.items():
    print(f"Cluster {cluster + 1} (length = {len(axes)}):")
    for axis in axes:
        print(f"  - {axis}")
    print("")  # New line for readability between clusters

/home/lisabdunlap/anaconda3/envs/summarizing/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


['   High: Specific, detailed information    Low: General, abstract overview', "   High: Emphasizes oxygen's role.    Low: Less focus on oxygen.", '   High: In-depth analysis and broad coverage.    Low: Single anecdotal focus.', '   High: Elaborate with solutions.    Low: Simplified facts.', '   High: Detailed Unit Examples    Low: Generalized Unit Examples', '   High: Cautious, conservative language    Low: Neutral, informative language', '   High: Interactive, conversational tone.    Low: Maintains a formal tone.', '   High: Specific medication details included.    Low: General overview without specifics.', '   High: Humorous, sarcastic    Low: Serious, straightforward', '   High: Personalized, empathetic language    Low: Formal and informative tone', '   High: Detailed, specific information.    Low: Condensed, general explanation.', '   High: Formal, technical language.    Low: Informal, personal language.', '   High: Flexible, politically driven approach    Low: Emphasis on clear e

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpod0i1w3u
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpod0i1w3u/_remote_module_non_scriptable.py
/home/lisabdunlap/anaconda3/envs/summarizing/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Batches: 100%|██████████| 11/11 [00:01<00:00,  6.37it/s]
/home/lisabdunlap/anaconda3/envs/summarizing/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  w

Cluster 1 (length = 7):
  - Focus on specific topics:    High: Individual candidate views    Low: General historical coverage
  - Interpretation of monotheistic and polytheistic beliefs:    High: Theological distinctions explained    Low: Focus on global presence
  - Perspective on expressing views:    High: Detailed candidate viewpoints    Low: Neutral historical context
  - Approach to a libertarian society:    High: Discusses extreme views, skepticism.    Low: Provides clear vision without extremes.
  - Emphasis on Tradition and Change:    High: In-depth analysis of viewpoints on tradition and change.    Low: General description without deeper contrasts.
  - Engagement with Counterarguments:    High: Extensive analysis of opposing views    Low: Limited consideration of counterarguments
  - Engagement with the question:    High: Directly addressing candidate views    Low: Neutral informative tone

Cluster 2 (length = 54):
  - Depth of Information:    High: In-depth analysis and broad

In [9]:
# cluster_axes_descriptions_prompt = ["""I am trying to explain differences in the behavior of two LLM's by comparing their outputs over a dataset of question answer tuples. I have several descriptions of axes of variation found with the meanings of what it means to be low and high on this axis. 

# {axes}

# Some of these axes of variations could be named incorrectly or redundant with other axes. Please return a numbered list of new axis descriptions that fit the low and high descriptions. Your new set of axes should be distinct so each of the above axes fit under exactly one of your new axes. Please ensure you descriptions of what is considered high and low on each axis is clear, concise, and easy for a human to understand.""", 
# """thanks! Now can you please categorize each of the original axes under you new list of axes? Remember that each original axis should only belong to one of the axes you described. Here are the original axes again for references:

# {axes}

# Please structure your response as:

# {{new axis}}:  High: {{new axis high description}} Low: {{new axis low description}}
# - {{original axis 1}}:  High: {{original axis high description}} Low: {{original axis low description}}
# - {{original axis 2}}:  High: {{original axis high description}} Low: {{original axis low description}}"""]
# smaller_systems_prompt = "You are a helpful assistant. Your outputs adhere to the format given by the user."

# cluster_1 = grouped_axes[3]
# prompt_1 = cluster_axes_descriptions_prompt[0].format(axes="\n".join(cluster_1))
# cluster_1_reduced_axes = get_llm_output(prompt_1, model="gpt-4", system_prompt=smaller_systems_prompt)
# print(cluster_1_reduced_axes)

In [10]:
# history = [{"role": "user", "content": prompt_1}, {"role": "assistant", "content": cluster_1_reduced_axes}]
# prompt_2 = cluster_axes_descriptions_prompt[1].format(axes="\n".join(cluster_1))
# cluster_1_reduced_axes_categorized = get_llm_output(prompt_2, model="gpt-4", system_prompt=smaller_systems_prompt, history=history)
# print(cluster_1_reduced_axes_categorized)

In [11]:
# conversion_prompt = """Below is a numbered list of axes of variation with their high and low descriptions, along with the original axes categorized under them. Please convert this list into a JSON format and return it.

# {axes}

# Please format you JSON response such that the keys are the axes of varation along with their high and low descriptions and the values are a list of the original axes with their high and low descriptions categorized under them. 

# An example JSON format is shown below:
# "{{new_axis_1_with_high_low}}" : ["{{original_axis_1_with_high_low}}", "{{original_axis_2_with_high_low}}"]

# I should be able to take this response directly and convert it into a Python object with ast.literal_eval().
# """
# converted_list = get_llm_output(conversion_prompt.format(axes=cluster_1_reduced_axes_categorized), model="gpt-3.5-turbo", system_prompt=smaller_systems_prompt)
# print(converted_list)

In [12]:
def get_cluster_axes(cluster):
    cluster_axes_descriptions_prompt = ["""The following are the axes of variation that you can consider when comparing the two model outputs along with a description of how two models (A and B) vary along that axis. Each axis has a name as well as a description of what it means to be low and high on this axis. Many of these axes of variations could be named incorrectly or redundant with other axes. I want to cluster these axes so that I can better understand the general patterns seen in these models without having to look through so many axes. Please cluster this large list of axes into a minimal set of parent axes that cover the entire axis list. Please ensure these parent axes' descriptions of what makes an item high or low on that axis align with the high and low descriptions of the axes they cover. Your new set of axes should be distinct so each of the above axes fit under exactly one of your new axes.
                        
    Here are the axes of varaiation (note each axis is formatted {{axis name}}: High: {{high description}} Low: {{low description}}):
    {axes}

    Again I want to cluster these axes into a minimal set of parent axes that cover the entire axis list. Please ensure these parent axes' descriptions of what makes an item high or low on that axis align with the high and low descriptions of the axes they cover. Your new set of axes should be distinct so each of the above axes fit under exactly one of your new axes. Please ensure each axis and parent axis contains an axis name and descriptions of what it means to score high or low on that axis in the same format as the provided axes.  Please ensure the descriptions of what is considered high and low on each axis is clear, concise, under 10 words. Please focus on patterns that are important for understanding the behavior of a language model, as these will later be used to help debug an important system""", 
                                     
    """thanks! Now can you please categorize each of the original axes under you new list of axes? Remember that each original axis should only belong to one of the axes you described. Here are the original axes again for reference:
    {axes}

    Please structure your response as a numbered list that adheres to the following format:

    1. {{new axis name}}:  High: {{new axis high description}} Low: {{new axis low description}}
    - {{original axis 1}}:  High: {{original axis high description}} Low: {{original axis low description}}
    - {{original axis 2}}:  High: {{original axis high description}} Low: {{original axis low description}}
    
    Please ensure that all the original axes above are categorized under the new axes you provide. Please ensure each of original axes listed above should only belong to one of the axes you described. If there are any axes that do not fit under any of the new axes you provided, please list them under a new axis. If there are any new axes that fit the same original axes, please merge them together."""]
    smaller_systems_prompt = "You are a helpful assistant. Your outputs adhere to the format given by the user."

    prompt_1 = cluster_axes_descriptions_prompt[0].format(axes="\n".join(cluster))
    cluster_1_reduced_axes = get_llm_output(prompt_1, model="gpt-4-0125-preview", system_prompt=smaller_systems_prompt)

    history = [{"role": "user", "content": prompt_1}, {"role": "assistant", "content": cluster_1_reduced_axes}]
    prompt_2 = cluster_axes_descriptions_prompt[1].format(axes="\n".join(cluster))
    cluster_1_reduced_axes_categorized = get_llm_output(prompt_2, model="gpt-4-0125-preview", system_prompt=smaller_systems_prompt, history=history)

    return prompt_1, prompt_2, cluster_1_reduced_axes, cluster_1_reduced_axes_categorized

def convert_axes_clusters_to_df(llm_output):
    conversion_prompt = """Below is a numbered list of axes of variation with their high and low descriptions, along with the original axes categorized under them. Please convert this list into a JSON format and return it.

    {axes}

    Please format you JSON response such that the keys are the axes of varation along with their high and low descriptions and the values are a list of the original axes with their high and low descriptions categorized under them. 

    An example JSON format is shown below:
    "{{new_axis_1_with_high_low}}" : ["{{original_axis_1_with_high_low}}", "{{original_axis_2_with_high_low}}"]

    Please ensure that the axes that make up the keys are copied verbatim from your previous output, including the high low descriptions. Please make sure that the original axes that make up the values are copied verbatim from the list above, including the high low descriptions. I should be able to take this response directly and convert it into a Python object with ast.literal_eval().
    """
    for i in range(3):
        try:
            converted_list = get_llm_output(conversion_prompt.format(axes=llm_output), model="gpt-3.5-turbo", system_prompt=smaller_systems_prompt)
            cluster_1_converted_list = ast.literal_eval(converted_list)
            # make sure high and low are mentioned in the keys
            for key in cluster_1_converted_list.keys():
                if "high" not in key.lower() and "low" not in key.lower():
                    print(f"ERROR in key: {key}")
                    raise ValueError("High or low not mentioned in the key")
            # Creating lists to store the processed data
            sub_axes_list = []
            axis_list = []

            # Looping through the dictionary to fill the lists
            for axis, sub_axes in cluster_1_converted_list.items():
                for sub_axis in sub_axes:
                    axis_list.append(axis)
                    sub_axes_list.append(sub_axis)

            # Creating a DataFrame from the lists
            return converted_list, cluster_1_converted_list, pd.DataFrame({
                'sub_axes': sub_axes_list,
                'axis': axis_list
            })
        except:
            print(f"Attempt {i + 1} failed. Retrying...")
            converted_list = get_llm_output(conversion_prompt.format(axes=llm_output), model="gpt-3.5-turbo", system_prompt=smaller_systems_prompt, cache=False)

In [13]:
axes = grouped_axes[1]
prompt_1, prompt_2, cluster_1_reduced_axes, cluster_1_reduced_axes_categorized = get_cluster_axes(axes)
print(cluster_1_reduced_axes_categorized)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1. Analytical Depth: High: Comprehensive, in-depth analysis. Low: Basic overview, limited details.
   - Depth of Information: High: In-depth analysis and broad coverage. Low: Single anecdotal focus.
   - Depth of analysis: High: Detailed and comprehensive exploration. Low: Limited scope and anecdotal.
   - Consideration of Consequences: High: Thorough analysis. Low: Limited focus.
   - Depth of Explanation: High: In-depth coverage with specifics. Low: Broad overview with generalities.
   - Content Depth: High: Detailed scenarios, implications. Low: General overview.
   - Depth of Analysis: High: Comprehensive overview, various types. Low: Focused on specific comparison.
   - Complexity of arguments: High: Detailed, multifaceted analysis. Low: Simplified explanation.
   - Level of Detail (All occurrences): Varies from detailed breakdowns to minimal, brief overviews.
   - Detail Level (All occurrences): Varies from comprehensive, in-depth information to basic overviews.
   - Complexity (

In [14]:
output_3, cluster_1_converted_list, df_cluster_1 = convert_axes_clusters_to_df(cluster_1_reduced_axes_categorized)
output_3

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'{\n    "Analytical Depth: High: Comprehensive, in-depth analysis. Low: Basic overview, limited details.": [\n        "Depth of Information: High: In-depth analysis and broad coverage. Low: Single anecdotal focus.",\n        "Depth of analysis: High: Detailed and comprehensive exploration. Low: Limited scope and anecdotal.",\n        "Consideration of Consequences: High: Thorough analysis. Low: Limited focus.",\n        "Depth of Explanation: High: In-depth coverage with specifics. Low: Broad overview with generalities.",\n        "Content Depth: High: Detailed scenarios, implications. Low: General overview.",\n        "Depth of Analysis: High: Comprehensive overview, various types. Low: Focused on specific comparison.",\n        "Complexity of arguments: High: Detailed, multifaceted analysis. Low: Simplified explanation.",\n        "Level of Detail (All occurrences): Varies from detailed breakdowns to minimal, brief overviews.",\n        "Detail Level (All occurrences): Varies from co

In [15]:
df_cluster_1

,sub_axes,axis
0,Depth of Information: High: In-depth analysis ...,"Analytical Depth: High: Comprehensive, in-dept..."
1,Depth of analysis: High: Detailed and comprehe...,"Analytical Depth: High: Comprehensive, in-dept..."
2,Consideration of Consequences: High: Thorough ...,"Analytical Depth: High: Comprehensive, in-dept..."
3,Depth of Explanation: High: In-depth coverage ...,"Analytical Depth: High: Comprehensive, in-dept..."
4,"Content Depth: High: Detailed scenarios, impli...","Analytical Depth: High: Comprehensive, in-dept..."
5,Depth of Analysis: High: Comprehensive overvie...,"Analytical Depth: High: Comprehensive, in-dept..."
6,"Complexity of arguments: High: Detailed, multi...","Analytical Depth: High: Comprehensive, in-dept..."
7,Level of Detail (All occurrences): Varies from...,"Analytical Depth: High: Comprehensive, in-dept..."
8,Detail Level (All occurrences): Varies from co...,"Analytical Depth: High: Comprehensive, in-dept..."
9,Complexity (Both occurrences): High: Explores ...,"Analytical Depth: High: Comprehensive, in-dept..."


In [ ]:

all_cluster_axes, all_df_cluster, llm_logs = [], [], {}
for cluster, axes in grouped_axes.items():
    # if cluster > 1:
    #     break
    prompt_1, prompt_2, cluster_axes_1, cluster_axes = get_cluster_axes(axes)
    output_3, parsed_output3, df_cluster = convert_axes_clusters_to_df(cluster_axes)
    llm_logs[cluster] = {"prompt_1": prompt_1, "prompt_2": prompt_2, "output_1": cluster_axes_1, "output_2": cluster_axes, "output_3": output_3, "parsed_output3": parsed_output3}
    df_cluster['cluster'] = cluster + 1
    all_cluster_axes.append(cluster_axes)
    all_df_cluster.append(df_cluster)
    print(f"Cluster {cluster + 1} (length = {len(axes)}) (df length = {len(df_cluster)}):")
    print("")  # New line for readability between clusters

df_cluster = pd.concat(all_df_cluster)

In [ ]:
all_cluster_axes, all_df_cluster, llm_logs = [], [], {}
for cluster, axes in grouped_axes.items():
    # if cluster > 1:
    #     break
    prompt_1, prompt_2, cluster_axes_1, cluster_axes = get_cluster_axes(axes)
    output_3, parsed_output3, df_cluster = convert_axes_clusters_to_df(cluster_axes)
    llm_logs[cluster] = {"prompt_1": prompt_1, "prompt_2": prompt_2, "output_1": cluster_axes_1, "output_2": cluster_axes, "output_3": output_3, "parsed_output3": parsed_output3}
    df_cluster['cluster'] = cluster + 1
    all_cluster_axes.append(cluster_axes)
    all_df_cluster.append(df_cluster)
    print(f"Cluster {cluster + 1} (length = {len(axes)}) (df length = {len(df_cluster)}):")
    print("")  # New line for readability between clusters

df_cluster = pd.concat(all_df_cluster)

In [179]:
for cluster, axes in grouped_axes.items():
    print(f"Cluster {cluster + 1} (length = {len(axes)}) (df length = {len(df_cluster[df_cluster['cluster'] == cluster+1])}):")

Cluster 1 (length = 19) (df length = 19):
Cluster 2 (length = 31) (df length = 31):
Cluster 3 (length = 42) (df length = 37):
Cluster 4 (length = 22) (df length = 22):
Cluster 5 (length = 26) (df length = 26):
Cluster 6 (length = 23) (df length = 22):
Cluster 7 (length = 28) (df length = 24):
Cluster 8 (length = 16) (df length = 17):
Cluster 9 (length = 90) (df length = 80):
Cluster 10 (length = 36) (df length = 32):


In [180]:
llm_outputs = pd.DataFrame(llm_logs).T
# llm_outputs.to_csv("llm_outputs_new_clustering_technique_all.csv", index=False)

In [181]:
# df_cluster.to_csv("clustered_axes_new_clustering_technique_all.csv", index=False)

In [182]:
exploaded_df = df.explode('axis_description')
new_df_cluster = df_cluster
new_df_cluster['axis_description'] = new_df_cluster['sub_axes']


from fuzzywuzzy import fuzz
# Fuzzy match function
def is_match(str1, str2, threshold=90):
    return fuzz.ratio(str1, str2) > threshold

# Create an empty DataFrame for storing matches
matches = []
merged_dataframe = []

# Iterate over rows in df1
for index1, row1 in exploaded_df.iterrows():
    flag = False
    for index2, row2 in new_df_cluster.iterrows():
        if is_match(row1['axis_description'], row2['axis_description']):
            if flag:
                print(f"Multiple matches found for {row1['axis_description']}")
            # If match found, append to the matches DataFrame
            matches.append({
                'DF1_index': index1,
                'DF2_index': index2,
                'DF1_axis_description': row1['axis_description'],
                'DF2_axis_description': row2['axis_description']
            })
            row1_sans = row1.drop('axis_description')
            merged_dataframe.append({**row1, **row2})
            flag = True

# Show the matches
matches = pd.DataFrame(matches)
len(matches), len(pd.DataFrame(merged_dataframe))

Multiple matches found for Ethical considerations: High: Explicit ethical discussions Low: Focus on practical aspects
Multiple matches found for Tone and Subjectivity: High: Nuanced, subjective approach Low: Neutral, objective tone


(282, 282)

In [183]:
df = pd.DataFrame(merged_dataframe)

In [184]:
print(df.iloc[0]["axes"])

['Complexity of language', 'Specificity of examples', 'Tone towards risk', 'Call to action']


In [194]:
prompt_1, prompt_2, cluster_1_reduced_axes, cluster_1_reduced_axes_categorized = get_cluster_axes(list(df.axis.unique()))

HERE
systems prompt  You are a helpful assistant. Your outputs adhere to the format given by the user.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


HERE
systems prompt  You are a helpful assistant. Your outputs adhere to the format given by the user.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [197]:
output_3, parsed_output3, df_cluster = convert_axes_clusters_to_df(cluster_1_reduced_axes_categorized)

HERE
systems prompt  You are a helpful assistant. Your outputs adhere to the format given by the user.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Attempt 1 failed. Retrying...
HERE
systems prompt  You are a helpful assistant. Your outputs adhere to the format given by the user.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


HERE
systems prompt  You are a helpful assistant. Your outputs adhere to the format given by the user.
LLM Cache Hit


In [198]:
parsed_output3

{'Language Complexity and Accessibility\nHigh: Technical, complex terminology\nLow: Simple, accessible language': ["Language and Terminology\nHigh: Technical, didactic, complex language\nLow: Simple, layman's terms, accessible language",
  'Use of Technical Terms vs. Analogical Approach\nHigh: Heavy on technical terms, formal explanations\nLow: Uses analogies and simple language for clarity',
  'Terminology & Language Complexity\nHigh: Advanced, technical terminology used\nLow: Simple, accessible language, no jargon',
  'Medical Terminology Use\nHigh: Uses detailed, specific medical terms\nLow: Utilizes layperson-friendly language, simple explanations',
  'Clarity and Simplicity of Explanation\nHigh: Simple, easily understandable language\nLow: Complex, technical language',
  'Accessibility & Clarity\nHigh: Relatable, clear, and easy to understand\nLow: Focuses on complex medical content, less clear'],
 'Depth and Breadth of Information\nHigh: Detailed, comprehensive exploration\nLow: 

In [202]:
new_column = {"index": [], "super_axis": []}

for index, row in df.iterrows():
    flag = False
    for key, value in parsed_output3.items():
        for v in value:
            if is_match(row['axis'], v):
                new_column["index"].append(index)
                new_column["super_axis"].append(key)
                flag = True
    if not flag:
        print(f"No match found for {row['axis']}")

No match found for **Use of Analogies and Visuals**: High: Frequent, relatable examples and visual analogies. Low: Lacks analogies and visual explanations
No match found for Educational and Instructional Approach: High: Didactic, explanatory, maintaining an educational tone Low: Casual, slightly informal, not strictly educational
No match found for **Use of Analogies and Visuals**: High: Frequent, relatable examples and visual analogies. Low: Lacks analogies and visual explanations
No match found for Complexity of Explanation & Information:
High: Technical details, examples, comprehensive. Low: Simplified, basic language and content.
No match found for Formality and Tone: High: Formal, professional, technical language Low: Casual, conversational, personalized tone
No match found for Scope & Focus of Content:
High: Wide application, solution-oriented, comprehensive scope. Low: Narrow focus, problem emphasis, basic functionality.
No match found for Formality and Tone: High: Formal, profe

,index,super_axis
0,0,Language Complexity and Accessibility\nHigh: T...
1,1,Depth and Breadth of Information\nHigh: Detail...
2,2,Scope and Complexity of Consideration\nHigh: M...
3,3,Practicality and Application Focus\nHigh: Acti...
4,4,Depth and Breadth of Information\nHigh: Detail...
...,...,...
225,277,"Tone and Approach\nHigh: Empathetic, nuanced o..."
226,278,Technological and Medical Specialization\nHigh...
227,279,Technological and Medical Specialization\nHigh...
228,280,Depth and Breadth of Information\nHigh: Detail...


In [221]:
new_columns = pd.DataFrame(new_column)
new_columns.index = new_columns["index"]
# merge the two dataframes, putting nan where no match was found
new_df = pd.merge(df, new_columns, left_index=True, right_index=True, how='left')

In [5]:
new_df = pd.read_csv("new_pipeline_intermediate_results/super_axes_new_clustering_technique_all.csv")

In [6]:
# only keep rows with the number of super_axis rows > 10
biggest_clusters = [k for k,v in new_df['super_axis'].value_counts().to_dict().items() if v > 10]
for b in biggest_clusters:
    print(f"{b}\n---------")

Depth and Breadth of Information
High: Detailed, comprehensive exploration
Low: General overview, limited specifics
---------
Explanation and Analytical Depth
High: In-depth analysis, technical breakdown
Low: Broad overview, simple explanation
---------
Tone and Approach
High: Empathetic, nuanced or critical tone
Low: Neutral, factual tone
---------
Technological and Medical Specialization
High: Detailed, specific medical/technical insights
Low: General principles, basic explanations
---------
Scope and Complexity of Consideration
High: Multifaceted implications, complex scenarios
Low: Direct effects, simplified scenarios
---------
Content Structuring and Clarity
High: Structured, technical breakdown
Low: Basic, simplified overview
---------
Language Complexity and Accessibility
High: Technical, complex terminology
Low: Simple, accessible language
---------
Engagement and Personalization
High: Interactive, personalized content
Low: Generalized, impersonal information
---------
Practica

In [7]:
biggest_clusters = [k for k,v in new_df['super_axis'].value_counts().to_dict().items() if v > 10]
new_df_filtered = new_df[new_df['super_axis'].isin(biggest_clusters)]

In [44]:
scoring = """I am trying to explain differences in the behavior of two LLM's (A and B) by comparing their outputs over a dataset of question answer tuples. I have of found axes of variation with the meanings of what it means to be low and high on this axis.

For the following question answer tuple, please score the two models on the following axis of variation found in the dataset. The axis of variation is as follows:
{axes}
Please provide a score of -1, 0, or 1 for both model A and model B on each axis. A score of -1 means that Model A is low on the axis and Model B is high, 0 means that both models are similar on the axis, and 1 means that Model A is high on the axis and Model B is low. Please provide a score for each axis.

Here is the question answer tuple:
{question}

Please return the score followed by an explanantion of your thought process in the format "Score={{score}} Reason={{reasoning}}". The reasoning should be a short explanation of why you gave the score you did for each axis.
"""

example_question = new_df.iloc[0]['prompt']
# turn biggest_clusters list into a string numbered list
biggest_clusters_str = "\n".join([f"{i+1}. {cluster}" for i, cluster in enumerate(biggest_clusters)])

scoring_prompt = scoring.format(axes=biggest_clusters_str, question=example_question)
scoring_output = get_llm_output(scoring_prompt, model="gpt-4", system_prompt=smaller_systems_prompt, cache=False)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [45]:
print(scoring_output)

1. Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics: 0
2. Explanation and Analytical Depth - High: In-depth analysis, technical breakdown, Low: Broad overview, simple explanation: -1
3. Tone and Approach - High: Empathetic, nuanced or critical tone, Low: Neutral, factual tone: 0
4. Technological and Medical Specialization - High: Detailed, specific medical/technical insights, Low: General principles, basic explanations: 0
5. Scope and Complexity of Consideration - High: Multifaceted implications, complex scenarios, Low: Direct effects, simplified scenarios: -1
6. Content Structuring and Clarity - High: Structured, technical breakdown, Low: Basic, simplified overview: 0
7. Language Complexity and Accessibility - High: Technical, complex terminology, Low: Simple, accessible language: 0
8. Engagement and Personalization - High: Interactive, personalized content, Low: Generalized, impersonal information: 1
9. Practicalit

In [46]:
from fuzzywuzzy import fuzz
# Fuzzy match function
def is_match(str1, str2, threshold=90):
    return fuzz.ratio(str1, str2) > threshold

def parse_scores(scoring_output, biggest_clusters):
    # Splitting the string into lines
    lines = scoring_output.strip().split('\n')

    # Creating an empty dictionary to store the parsed scores
    parsed_dict_corrected = {}

    for line in lines:
        print(line)
        # Splitting by the last colon to correctly separate the description from the score
        key, value = line.replace("\n", "").rsplit(":", 1)
        key = key[3:].strip()  # Removing the numbering and extra spaces
        value = int(value.strip())  # Converting the score to an integer
        # match the key to the biggest_clusters via fuzzy matching
        flag = False
        for cluster in biggest_clusters:
            print(key, cluster)
            if is_match(key, cluster, threshold=85):
                parsed_dict_corrected[cluster] = value
                flag = True
        assert flag, f"No match found for {key}"
        
        # # Adding the corrected key-value pair to the dictionary
        # parsed_dict_corrected[key] = value

    return parsed_dict_corrected

In [47]:
parse_scores(scoring_output, biggest_clusters)

1. Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics: 0
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Depth and Breadth of Information
High: Detailed, comprehensive exploration
Low: General overview, limited specifics
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Explanation and Analytical Depth
High: In-depth analysis, technical breakdown
Low: Broad overview, simple explanation
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Tone and Approach
High: Empathetic, nuanced or critical tone
Low: Neutral, factual tone
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Technological and Medical Specialization
High: Detailed, specific medical/tec

{'Depth and Breadth of Information\nHigh: Detailed, comprehensive exploration\nLow: General overview, limited specifics': 0,
 'Explanation and Analytical Depth\nHigh: In-depth analysis, technical breakdown\nLow: Broad overview, simple explanation': -1,
 'Tone and Approach\nHigh: Empathetic, nuanced or critical tone\nLow: Neutral, factual tone': 0,
 'Technological and Medical Specialization\nHigh: Detailed, specific medical/technical insights\nLow: General principles, basic explanations': 0,
 'Scope and Complexity of Consideration\nHigh: Multifaceted implications, complex scenarios\nLow: Direct effects, simplified scenarios': -1,
 'Content Structuring and Clarity\nHigh: Structured, technical breakdown\nLow: Basic, simplified overview': 0,
 'Language Complexity and Accessibility\nHigh: Technical, complex terminology\nLow: Simple, accessible language': 0,
 'Engagement and Personalization\nHigh: Interactive, personalized content\nLow: Generalized, impersonal information': 1,
 'Practicality

In [48]:
results = pd.read_csv('results.csv')
# Step 1: Split the 'prompt' into 'question' and 'answers'
results['question'] = results['prompt'].apply(lambda x: x.split("\nModel A: ")[0])
results['answers'] = results['prompt'].apply(lambda x: x.split("\nModel A: ")[1])

# Step 2: Further split 'answers' into 'Model A' and 'Model B'
results['answer_a'] = results['answers'].apply(lambda x: x.split("\nModel B ")[0].strip("[]").replace("'", ""))
results['answer_b'] = results['answers'].apply(lambda x: x.split("\nModel B: ")[1].strip("[]").replace("'", ""))

# Clean up
results.drop('answers', axis=1, inplace=True)  # Remove the intermediate 'answers' column

# ask for the score for each prompt in results
scores = {c: [] for c in biggest_clusters}
scores_df = []

for i, row in results.iterrows():
    for _ in range(3):
        try:
            scoring_prompt = scoring.format(axes=biggest_clusters_str, question=row['prompt'])
            scoring_output = get_llm_output(scoring_prompt, model="gpt-4", system_prompt=smaller_systems_prompt)
            parsed_scores = parse_scores(scoring_output, biggest_clusters)
            for c in biggest_clusters:
                new_row = row.to_dict()
                scores[c].append(parsed_scores[c])
                new_row['super_axis'] = c
                new_row['score'] = parsed_scores[c]
                scores_df.append(new_row)
            break
        except:
            print(f"Attempt {_ + 1} failed. Retrying...")
            scoring_output = get_llm_output(scoring_prompt, model="gpt-4", system_prompt=smaller_systems_prompt, cache=False)
            

LLM Cache Hit
1. Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics: 0
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Depth and Breadth of Information
High: Detailed, comprehensive exploration
Low: General overview, limited specifics
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Explanation and Analytical Depth
High: In-depth analysis, technical breakdown
Low: Broad overview, simple explanation
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Tone and Approach
High: Empathetic, nuanced or critical tone
Low: Neutral, factual tone
Depth and Breadth of Information - High: Detailed, comprehensive exploration, Low: General overview, limited specifics Technological and Medical Specialization
High: Detailed, specif

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1. Depth and Breadth of Information
Attempt 1 failed. Retrying...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM Cache Hit
1. Depth and Breadth of Information (High: Detailed, comprehensive exploration - Low: General overview, limited specifics): 1
Depth and Breadth of Information (High: Detailed, comprehensive exploration - Low: General overview, limited specifics) Depth and Breadth of Information
High: Detailed, comprehensive exploration
Low: General overview, limited specifics
Depth and Breadth of Information (High: Detailed, comprehensive exploration - Low: General overview, limited specifics) Explanation and Analytical Depth
High: In-depth analysis, technical breakdown
Low: Broad overview, simple explanation
Depth and Breadth of Information (High: Detailed, comprehensive exploration - Low: General overview, limited specifics) Tone and Approach
High: Empathetic, nuanced or critical tone
Low: Neutral, factual tone
Depth and Breadth of Information (High: Detailed, comprehensive exploration - Low: General overview, limited specifics) Technological and Medical Specialization
High: Detailed, s

...

1. Depth and Breadth of Information
Attempt 1 failed. Retrying...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LLM Cache Hit
1. Depth and Breadth of Information: High: Detailed, comprehensive exploration Low: General overview, limited specifics: 0
Depth and Breadth of Information: High: Detailed, comprehensive exploration Low: General overview, limited specifics Depth and Breadth of Information
High: Detailed, comprehensive exploration
Low: General overview, limited specifics
Depth and Breadth of Information: High: Detailed, comprehensive exploration Low: General overview, limited specifics Explanation and Analytical Depth
High: In-depth analysis, technical breakdown
Low: Broad overview, simple explanation
Depth and Breadth of Information: High: Detailed, comprehensive exploration Low: General overview, limited specifics Tone and Approach
High: Empathetic, nuanced or critical tone
Low: Neutral, factual tone
Depth and Breadth of Information: High: Detailed, comprehensive exploration Low: General overview, limited specifics Technological and Medical Specialization
High: Detailed, specific medical

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1. Depth and Breadth of Information (High: Detailed, comprehensive exploration Low: General overview, limited specifics): 0
Depth and Breadth of Information (High: Detailed, comprehensive exploration Low: General overview, limited specifics) Depth and Breadth of Information
High: Detailed, comprehensive exploration
Low: General overview, limited specifics
Depth and Breadth of Information (High: Detailed, comprehensive exploration Low: General overview, limited specifics) Explanation and Analytical Depth
High: In-depth analysis, technical breakdown
Low: Broad overview, simple explanation
Depth and Breadth of Information (High: Detailed, comprehensive exploration Low: General overview, limited specifics) Tone and Approach
High: Empathetic, nuanced or critical tone
Low: Neutral, factual tone
Depth and Breadth of Information (High: Detailed, comprehensive exploration Low: General overview, limited specifics) Technological and Medical Specialization
High: Detailed, specific medical/technica

In [49]:
pd.DataFrame(scores_df).to_csv("new_pipeline_intermediate_results/scores.csv", index=False)